In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch 
print(torch.__version__)
import torch.nn as nn
import math

2.0.0


![](https://pic4.zhimg.com/80/v2-f476fbc9bc3a39d2fa65707eb0a93417_720w.webp)



# 整体结构
左边 encoder
1. MultiHead Attention 都包含add残差&norm归一化
2. MLP 

右边 decoder
1. Masked MultiHead Attention
2. MultiHead Attention 都包含add残差&norm归一化
3. MLP 

# 1.输入：embedding层

In [3]:
class Embeddings(nn.Module):
    def __init__(self,d_model,vocab):
        """
        构造函数：
        d_model：维度
        vocab：词表
        """
        super(Embeddings,self).__init__()  #这里确保继承父类
        self.lut = nn.Embedding(vocab,d_model) #词嵌入对象lut
        self.d_model = d_model  #得到维度
    def forward(self,x):
        """
        Embedding层前向传播
        x:word经过词表之后的 one-hot向量
        """
        embedds = self.lut(x) #词嵌入
        return embedds * math.sqrt(self.d_model) #实现值放大，因为dimmension对其进行了稀释
        

In [4]:
# # 测试阶段

# import torch
# import torch.nn as nn

# # 定义词汇表大小和词嵌入维度
# vocab_size = 10000
# embedding_dim = 300

# # 创建一个词嵌入层
# embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# # 生成一个输入序列（假设每个句子最大长度为10）
# input_sequence = torch.LongTensor([[1, 5, 4, 3, 2, 9, 8, 7, 6, 0]])
# print(input_sequence.shape)

# # 将输入序列传递给词嵌入层
# embedded_output = embedding_layer(input_sequence)
# embedds = embedded_output * math.sqrt(embedding_dim)
# print(embedded_output)
# print(embedded_output.shape)  # 输出嵌入后的张量形状
# sum = torch.sum(embedded_output,dim = 2) #后面决定在 哪个维度 求和
# print(sum) #看上去 和不确定
# print(sum.shape)

# print("处理后的embedds: \n",embedds)



# 2.位置编码 Positional Encodding